In [14]:
import pandas as pd
from utils import load_csv
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
from numpy import nan as Nan
from numpy import inf as inf
from tqdm import tqdm
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_predict
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import SelectFromModel

from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from data_preprocessing import *
from scoring import bootstrap_roc_auc
from catboost import CatBoostRanker, Pool
from copy import deepcopy
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

In [4]:
group_names = np.loadtxt('ranking_pair_comparing/group_names.txt', dtype=str, delimiter=',')

In [5]:
df = pd.read_csv('exported.predictions.Kahle2020.csv')
groups = load_csv(f"groups_and_oxi_states_5_frames/df_features_step_0.pkl")
df = df.merge(groups, left_on=['src_id'], right_on=['stru_id'])
df = df.drop(['starting_structure', 'src_database', 'src_id'], axis = 1)
df['group_id'] = 1
# X, y = df.drop(['stru_label', 'stru_id', 'is_good', 'fv_1p0_WARNlowPES', 'fv_2p0_WARNlowPES', 'fv_3p0_WARNlowPES', 'fv_4p0_WARNlowPES'], axis = 1), df['is_good'].astype(int).to_numpy()

In [6]:
df['relevance'] = -1
df.sort_values(by='diffusion_mean_cm2_s', ascending=False, inplace = True)

In [7]:
len_A = len(df[df['group'] == 'group_A'])
len_B = len(df[df['group'] == 'group_B'])
len_C = len(df[df['group'] == 'group_C'])
len_E = len(df[df['group'] == 'group_E'])

In [8]:
df.loc[df['group'] == 'group_A', 'relevance'] = np.arange(len_A)
df.loc[df['group'] == 'group_B', 'relevance'] = len_A + np.arange(len_B)
df.loc[df['group'] == 'group_E', 'relevance'] = len_A+len_B + np.arange(len_E)
df.loc[df['group'] == 'group_C', 'relevance'] = len_A+len_B+len_E + np.arange(len_C)

In [9]:
df['relevance'] = df.shape[0] - df['relevance']

In [10]:
import catboost
X, y = df[['fv_1p0_disconnected',
 'fv_1p0_connected',
 'fv_1p0_WARNlowPES',
 'fv_2p0_disconnected',
 'fv_2p0_connected',
 'fv_2p0_WARNlowPES',
 'fv_3p0_disconnected',
 'fv_3p0_connected',
 'fv_3p0_WARNlowPES',
 'fv_4p0_disconnected',
 'fv_4p0_connected',
 'fv_4p0_WARNlowPES',
 'barrier']], df['relevance'].astype(int).to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

train_pool = catboost.Pool(X_train, y_train, group_id = df['group_id'].to_list()[:len(y_train)])
test_pool = catboost.Pool(X_test, y_test, group_id = df['group_id'].to_list()[len(y_train):])

In [11]:
default_parameters = {
    'iterations': 500,
    'custom_metric': ['NDCG', 'PFound', 'AverageGain:top=10'],
    'verbose': False,
    'random_seed': 0,
}

parameters = {}

In [16]:
def fit_model(loss_function, additional_params, train_pool, test_pool = None):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True)
    
    return model

In [15]:
df.index = np.arange(0, len(df))

group_names = np.array(np.zeros((len(df), len(df))), dtype=str)
group_scores = np.array(np.zeros((len(df), len(df))))

X, y = df[['fv_1p0_disconnected',
 'fv_1p0_connected',
 'fv_1p0_WARNlowPES',
 'fv_2p0_disconnected',
 'fv_2p0_connected',
 'fv_2p0_WARNlowPES',
 'fv_3p0_disconnected',
 'fv_3p0_connected',
 'fv_3p0_WARNlowPES',
 'fv_4p0_disconnected',
 'fv_4p0_connected',
 'fv_4p0_WARNlowPES',
 'barrier']], df['relevance'].astype(int).to_numpy()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.33, random_state=42)

train_pool = catboost.Pool(X_train, y_train, group_id = df['group_id'].to_list()[:len(y_train)])
test_pool = catboost.Pool(X_test, y_test, group_id = df['group_id'].to_list()[len(y_train):])

for i in tqdm(range(len(df))):
    for j in tqdm(range(i + 1, len(df))):
        print(i, j)
        X_train = X_scaled[~np.isin(np.arange(len(X_scaled)), [i, j])]
        y_train = y[~np.isin(np.arange(len(y)), [i, j])]
        group_ids = df['group_id'][~np.isin(np.arange(len(df['group_id'])), [i, j])]
        train_pool = catboost.Pool(X_train, y_train, group_id = group_ids)

        X_test = X_scaled[np.isin(np.arange(len(X_scaled)), [i, j])]
        y_test = y[np.isin(np.arange(len(y)), [i, j])]
        group_ids = df['group_id'][np.isin(np.arange(len(df['group_id'])), [i, j])]
        test_pool = catboost.Pool(X_test, y_test, group_id = group_ids)

        model = catboost.CatBoostRanker(loss_function='PairLogit', verbose = False)
        model.fit(train_pool, plot=False)
        preds = model.predict(test_pool)
        preds = np.exp(preds) / (1 + np.exp(preds))
        
        first_group = df.iloc[i]['group'][6:]
        second_group = df.iloc[j]['group'][6:]
        first_score = df.iloc[i]['relevance']
        second_score = df.iloc[j]['relevance']

        group_names[i, j] = ''.join(sorted([first_group, second_group]))

        if (preds[0] >= preds[1] and first_score >= second_score) or (preds[1] >= preds[0] and second_score >= first_score):
            group_scores[i, j] = 1
        else:
            group_scores[i, j] = -1


  0%|          | 0/116 [00:00<?, ?it/s]

0 1


0 2


  0%|          | 0/116 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [17]:
d = {}
for i, row_i in df.iterrows():
    for j, row_j in df.iterrows():
        
        train_pool = catboost.Pool(X_train, y_train, group_id = df['group_id'].to_list()[:len(y_train)])
        model = catboost.CatBoostRanker(loss_function='PairLogit', verbose = True)
        model.fit(train_pool, plot=False)
        print(sorted((row_i['group'][6:], row_j['group'][6:])))
        # d[sorted((row_i['group'][6:], row_j['group'][6:]))]

0:	learn: 0.6844461	total: 15.2ms	remaining: 15.2s
1:	learn: 0.6758457	total: 26.8ms	remaining: 13.4s
2:	learn: 0.6700600	total: 37.7ms	remaining: 12.5s
3:	learn: 0.6648553	total: 48.2ms	remaining: 12s
4:	learn: 0.6586940	total: 58.6ms	remaining: 11.7s
5:	learn: 0.6535140	total: 69ms	remaining: 11.4s
6:	learn: 0.6475490	total: 79.2ms	remaining: 11.2s
7:	learn: 0.6421472	total: 89.3ms	remaining: 11.1s
8:	learn: 0.6355843	total: 99.3ms	remaining: 10.9s
9:	learn: 0.6299388	total: 109ms	remaining: 10.8s
10:	learn: 0.6267081	total: 119ms	remaining: 10.7s
11:	learn: 0.6214522	total: 128ms	remaining: 10.5s
12:	learn: 0.6136709	total: 137ms	remaining: 10.4s
13:	learn: 0.6051011	total: 147ms	remaining: 10.3s
14:	learn: 0.6024348	total: 156ms	remaining: 10.2s
15:	learn: 0.5972222	total: 165ms	remaining: 10.2s
16:	learn: 0.5946247	total: 174ms	remaining: 10.1s
17:	learn: 0.5917629	total: 184ms	remaining: 10s
18:	learn: 0.5892482	total: 193ms	remaining: 9.95s
19:	learn: 0.5842927	total: 202ms	rema

KeyboardInterrupt: 

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
cb_ranker = catboost.CatBoostRanker(loss_function='PairLogit', verbose = True)
cb_ranker.fit(train_pool, plot=False)

0:	learn: 0.6842463	total: 57.9ms	remaining: 57.9s
1:	learn: 0.6765266	total: 66ms	remaining: 32.9s
2:	learn: 0.6720218	total: 73.2ms	remaining: 24.3s
3:	learn: 0.6640322	total: 80.4ms	remaining: 20s
4:	learn: 0.6557389	total: 87.2ms	remaining: 17.4s
5:	learn: 0.6513764	total: 94.1ms	remaining: 15.6s
6:	learn: 0.6432371	total: 101ms	remaining: 14.3s
7:	learn: 0.6375288	total: 108ms	remaining: 13.3s
8:	learn: 0.6304411	total: 114ms	remaining: 12.6s
9:	learn: 0.6238003	total: 121ms	remaining: 12s
10:	learn: 0.6165087	total: 128ms	remaining: 11.5s
11:	learn: 0.6128803	total: 134ms	remaining: 11s
12:	learn: 0.6022970	total: 140ms	remaining: 10.7s
13:	learn: 0.5993746	total: 147ms	remaining: 10.3s
14:	learn: 0.5945906	total: 153ms	remaining: 10.1s
15:	learn: 0.5900348	total: 159ms	remaining: 9.79s
16:	learn: 0.5873735	total: 166ms	remaining: 9.58s
17:	learn: 0.5796138	total: 172ms	remaining: 9.39s
18:	learn: 0.5724565	total: 179ms	remaining: 9.22s
19:	learn: 0.5698932	total: 185ms	remaining

In [28]:
pred = cb_ranker.predict(test_pool)
np.exp(pred)/(1 + np.exp(pred))

array([ 1.82093661,  1.50485443, -3.86087461,  1.12231243, -0.63303477,
        2.31743267,  2.91543963,  2.96336258, -1.54148184,  3.01916231,
       -2.89681766, -6.03874524,  2.05138681, -0.06817592, -2.28916179,
        8.08637359,  1.30550483, -1.04587445, -5.49504594, -0.82322387,
       -0.90620666, -3.95920003, -5.34730026, -3.71459208,  7.49606445,
       -5.30713515,  0.53971322,  2.53276989, -3.67048848,  1.45261619,
        1.48465482, -2.27976243, -1.25470509,  4.3645889 , -0.18680791,
        3.02402884,  2.92084303, -3.02858122,  3.53699122])

In [26]:
cb_ranker.predict(test_pool)

array([ 0.65416887,  0.45137547, -0.46905568,  0.81027349,  0.39198095,
        0.12469002,  0.93329311,  0.8008096 , -0.47276234,  0.52690676,
        0.02282159, -0.80657076,  0.176174  , -0.32516929, -0.44450787,
        1.07675349,  0.34440615, -0.53600604, -0.6331056 , -0.46067288,
       -0.4892162 , -0.77595043, -0.72085581, -0.35352004,  1.40079404,
       -0.97472004,  0.45775288,  0.98607656, -0.54368559,  0.72405953,
        0.13668476, -0.47079284, -0.5822944 ,  0.88653496, -0.51029972,
        0.80045125,  0.89560306, -0.56150007,  0.59481   ])

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    train_pool = catboost.Pool(X_scaled[train_index, :], y[train_index], group_id = df['group_id'][train_index].to_list())
    cb_ranker = catboost.CatBoostRanker(loss_function='PairLogit', verbose = False)
    cb_ranker.fit(train_pool)
    preds[test_index] = cb_ranker.predict(X_scaled[test_index])

In [13]:
loo = LeaveOneOut()
loo.get_n_splits(X_scaled)
preds = np.zeros(len(y))
for i, (train_index, test_index) in enumerate(tqdm(loo.split(X))):
    train_pool = catboost.Pool(X_scaled[train_index, :], y[train_index], group_id = df['group_id'][train_index].to_list())
    cb_ranker = catboost.CatBoostRanker(loss_function='PairLogit', verbose = False)
    cb_ranker.fit(train_pool)
    preds[test_index] = cb_ranker.predict(X_scaled[test_index])

114it [17:43,  8.83s/it]

In [71]:
roc_auc_score(y, preds)

0.7574534161490684

In [23]:
import pandas as pd
from utils import load_csv
import numpy as np
import os
import pickle
import numpy as np
import pandas as pd
from numpy import nan as Nan
from numpy import inf as inf
from tqdm import tqdm
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_predict
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import SelectFromModel

from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from data_preprocessing import *
from scoring import bootstrap_roc_auc
from catboost import CatBoostRanker, Pool
from copy import deepcopy
import numpy as np
import os
import pandas as pd 
import catboost


if __name__ == "__main__":

    df = pd.read_csv('exported.predictions.Kahle2020.csv')
    groups = load_csv(f"groups_and_oxi_states_5_frames/df_features_step_0.pkl")
    df = df.merge(groups, left_on=['src_id'], right_on=['stru_id'])
    df = df.drop(['starting_structure', 'src_database', 'src_id'], axis = 1)
    df['group_id'] = 1
    df['relevance'] = -1
    df.sort_values(by='diffusion_mean_cm2_s', ascending=False, inplace = True)
    len_A = len(df[df['group'] == 'group_A'])
    len_B = len(df[df['group'] == 'group_B'])
    len_C = len(df[df['group'] == 'group_C'])
    len_E = len(df[df['group'] == 'group_E'])
    df.loc[df['group'] == 'group_A', 'relevance'] = np.arange(len_A)
    df.loc[df['group'] == 'group_B', 'relevance'] = len_A + np.arange(len_B)
    df.loc[df['group'] == 'group_E', 'relevance'] = len_A+len_B + np.arange(len_E)
    df.loc[df['group'] == 'group_C', 'relevance'] = len_A+len_B+len_E + np.arange(len_C)
    df['relevance'] = df.shape[0] - df['relevance']
    
    df.index = np.arange(0, len(df))

    group_names = []
    group_scores = []
    results = list()
    # group_names = np.loadtxt('ranking_pair_comparing/group_names.txt', dtype = str, delimiter=',')
    # group_scores = np.loadtxt('ranking_pair_comparing/group_scores.txt', dtype = int, delimiter=',')

    X, y = df[['fv_1p0_disconnected',
        'fv_1p0_connected',
        'fv_1p0_WARNlowPES',
        'fv_2p0_disconnected',
        'fv_2p0_connected',
        'fv_2p0_WARNlowPES',
        'fv_3p0_disconnected',
        'fv_3p0_connected',
        'fv_3p0_WARNlowPES',
        'fv_4p0_disconnected',
        'fv_4p0_connected',
        'fv_4p0_WARNlowPES',
        'barrier']], df['relevance'].astype(int).to_numpy()

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pairs = np.random.randint(low = 0, high= len(X_scaled), size=(100, 2), dtype=int)
    for [i, j] in pairs:
        print(i, j)
        X_train = X_scaled[~np.isin(np.arange(len(X_scaled)), [i, j])]
        y_train = y[~np.isin(np.arange(len(y)), [i, j])]
        group_ids = df['group_id'][~np.isin(np.arange(len(df['group_id'])), [i, j])]
        train_pool = catboost.Pool(X_train, y_train, group_id = group_ids)

        X_test = X_scaled[np.isin(np.arange(len(X_scaled)), [i, j])]
        y_test = y[np.isin(np.arange(len(y)), [i, j])]
        group_ids = df['group_id'][np.isin(np.arange(len(df['group_id'])), [i, j])]
        test_pool = catboost.Pool(X_test, y_test, group_id = group_ids)

        model = catboost.CatBoostRanker(loss_function='PairLogitPairwise', verbose = False)
        model.fit(train_pool, plot=False)
        preds = model.predict(test_pool)
        preds = np.exp(preds) / (1 + np.exp(preds))
        
        first_group = df.iloc[i]['group'][6:]
        second_group = df.iloc[j]['group'][6:]
        first_score = df.iloc[i]['relevance']
        second_score = df.iloc[j]['relevance']


        results.append([''.join(sorted([first_group, second_group])), ])
        print(''.join(sorted([first_group, second_group])))
        # group_names[i, j] = ''.join(sorted([first_group, second_group]))
        group_names.append(''.join(sorted([first_group, second_group])))

        if (preds[0] >= preds[1] and first_score >= second_score) or (preds[1] >= preds[0] and second_score >= first_score):
            group_scores.append([1, preds[0], preds[1], first_score, second_score])
        else:
            group_scores.append([-1, preds[0], preds[1], first_score, second_score])

        # print(preds[0], preds[1])
        # print(first_score, second_score)

        with open("ranking_pair_comparing/group_scores.txt", "w") as output:
            output.write(str(group_scores))

        with open("ranking_pair_comparing/group_names.txt", "w") as output:
            output.write(str(group_names))

        # np.savetxt('ranking_pair_comparing/group_scores.txt', group_scores, fmt = '%d', delimiter=',')
        # np.savetxt('ranking_pair_comparing/group_names.txt', group_names, fmt = '%s', delimiter=',')
            # with open("ranking_pair_comparing/group_names.txt", "w") as output:
            #     output.write(str(group_names))

            # with open("ranking_pair_comparing/group_scores.txt", "w") as output:
            #     output.write(str(group_scores))




116
[[104  90]
 [103  54]
 [ 51  20]
 [ 59  24]
 [104  22]
 [ 51  11]
 [ 97  95]
 [ 38  69]
 [ 72  37]
 [ 86   9]
 [ 74 102]
 [  0   1]
 [ 70  80]
 [ 62  44]
 [ 11  59]
 [ 53  93]
 [108   4]
 [  8  85]
 [ 92  50]
 [ 35  20]
 [  1  14]
 [ 99  47]
 [111  31]
 [  9 101]
 [106  50]
 [ 28 106]
 [106  54]
 [ 17   8]
 [ 90  70]
 [ 86  90]
 [ 95  55]
 [ 55  50]
 [ 76  31]
 [ 20 115]
 [ 33  73]
 [ 59  29]
 [ 11  76]
 [ 11  28]
 [ 30  14]
 [ 45  18]
 [ 23  53]
 [114  70]
 [ 14   4]
 [ 54  53]
 [ 40  44]
 [ 18   0]
 [ 82  34]
 [ 92  63]
 [ 41  41]
 [ 61  37]
 [ 67   9]
 [  8  76]
 [  8  99]
 [ 78  31]
 [ 27  22]
 [ 99  11]
 [ 59  70]
 [ 46  42]
 [ 67  67]
 [111  11]
 [ 79 100]
 [ 21  11]
 [ 81  82]
 [ 39  94]
 [ 33  76]
 [ 71  90]
 [ 51   1]
 [ 25  95]
 [ 74  97]
 [ 65  19]
 [ 44  61]
 [ 34 111]
 [ 37 105]
 [110  45]
 [ 49  56]
 [ 86  63]
 [ 68  84]
 [ 63  93]
 [  6  23]
 [ 96  14]
 [114  63]
 [ 75  39]
 [105  22]
 [ 36  68]
 [ 75  56]
 [  4 115]
 [ 70  37]
 [ 20  40]
 [ 18  31]
 [ 71  73]
 [ 56 

KeyboardInterrupt: 